# Demo CNN for Wildfire Growth Prediction

Below is starter code for a cnn solution to solve the wildfire growth challenge!

We provide infrastructure and helper functions to call and process the data.

It is up to your team to fill in necessary blanks and improve the pipeline.

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and set to:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead.")

GPU is available and set to: Tesla T4


#### Pip Install

In [ ]:
!pip install rasterio matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/MAG_Wildfire_Hackathon/Wildfire_Hackathon_Complete'

/content/drive/.shortcut-targets-by-id/1MNSgACFZBntIhsetYbiUztFCcTtX8ZoD/MAG_Wildfire_Hackathon/Wildfire_Hackathon_Complete


In [ ]:
!ls

'0.load data -- Aania edits.ipynb'   demo-MVP-bilaledits.ipynb	 hotspots   train
'0. load data.ipynb'		     demo-MVP-maanav.ipynb	 output
'1. demo-MVP.ipynb'		     fbp_lookup.csv		 test


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

# Paths for data and fires
data_path = "./"
train_path = data_path + "train/"
test_path = data_path + "test/"
tr_fnums = ["fire1209", "fire1298", "fire1386", "fire2034", "fire2210", "fire2211", "fire2212"]
te_fnums = ["fire2214"]

In [ ]:
# Util variables
device = 'cuda'
target_shape = (528, 720)

# Util functions
def pad_to_fit(d, shape):
    h, w = d.shape
    pad_h = shape[0] - h
    pad_w = shape[1] - w
    if pad_h > 0 or pad_w > 0:
        pad_top = pad_h // 2
        pad_bottom = pad_h - pad_top
        pad_left = pad_w // 2
        pad_right = pad_w - pad_left

        d = np.pad(d, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    return d

def normalize(d):
    m = np.mean(d)
    s = np.std(d)
    return (d - m)/s

def tif2np(tif):
    with rio.open(tif) as src:
        data = src.read(1)  # Read the first band
    return pad_to_fit(np.nan_to_num(data, nan=0.0), target_shape)

## Functions to load data

The load fire function loads and processes data for the denoted fire. The fire is then stacked into a numpy array.

The load day function loads in a day of data for a specified fire.

<ins>**Additional data should be loaded and specified into this function**.<ins>

In [21]:
import geopandas as gpd

fire1209_shp = gpd.read_file('/content/drive/MyDrive/MAG_Wildfire_Hackathon/Wildfire_Hackathon_Complete/train/fire1209/fire/fire1209.shp')
# fire1209.ignition

In [22]:
fire1209_shp

,DOB,geometry
0,171,"POLYGON ((-1840010.000 1019920.000, -1839810.0..."
1,198,"MULTIPOLYGON (((-1825610.000 998220.000, -1825..."
2,199,"MULTIPOLYGON (((-1825110.000 998220.000, -1825..."
3,208,"MULTIPOLYGON (((-1865010.000 1025920.000, -186..."
4,209,"MULTIPOLYGON (((-1863010.000 1027020.000, -186..."
5,210,"MULTIPOLYGON (((-1860610.000 1028320.000, -186..."
6,211,"MULTIPOLYGON (((-1863410.000 1028120.000, -186..."
7,213,"MULTIPOLYGON (((-1868710.000 1025420.000, -186..."
8,214,"MULTIPOLYGON (((-1862710.000 1029620.000, -186..."
9,216,"MULTIPOLYGON (((-1860710.000 1029220.000, -186..."


In [ ]:

def load_day(path, day):
    # fire_weather
    fwi = path+'/fire_weather/fire_weather_index_day{}.tif'.format(day)
    fwi = normalize(tif2np(fwi))
    # weather relative humidity
    wrh = path+'/weather/noon_relative_humidity_day{}.tif'.format(day)
    wrh = normalize(tif2np(wrh))
    # weather wind speed
    wws = path+'/weather/noon_wind_speed_day{}.tif'.format(day)
    wws = normalize(tif2np(wws))
    # Add more data here
    #...

    return [fwi, wrh, wws]

def load_fire(fire_num, split = "Train"):
    path = train_path + fire_num
    if split == "Test":
        path = test_path + fire_num

    ftif = path + "/fire/{}.tif".format(fire_num)
    if split == "Test":
        ftif = path + "/fire/{}_train.tif".format(fire_num)
    fdata = tif2np(ftif)

    minjd, maxjd = int(np.min(fdata[np.nonzero(fdata)])), int(np.max(fdata))
    lastjd = maxjd
    if split == "Test":
        maxjd += 21

    elev = normalize(tif2np(path+'/topography/dem.tif'))
    slope = normalize(tif2np(path+'/topography/slope.tif'))
    fuels = tif2np(path+'/fuels/fbp_fuels.tif')
    ignition = tif2np(path+'/fire/ignitions.tif')


    dataset = []
    gt = ignition
    cfire = ignition
    for d in range(minjd, maxjd):
        data = {}

        fuels[cfire != 0] = 0
        ft = [fuels]
        ft.extend([cfire, gt, slope, elev])
        ft.extend(load_day(path, d))
        ft = np.stack(ft)
        data['ft'] = ft

        if d < lastjd:
            gt = fdata == float(d)
            data['gt'] = gt

        cfire = np.logical_or(cfire ,gt)

        dataset.append(data)
    return dataset

## Create the datasets and dataloaders

<ins>Create/implement data augmentations/transformations here<ins>

## Define the network/model

In this example, we define a simple 2 layer cnn model.

<ins>**Modify the model as you see fit!**<ins>

In [ ]:
class FireDataset(Dataset):
    def __init__(self, split="Train"):
        fnums = tr_fnums if split=="Train" else te_fnums
        self.dataset = []
        for fnum in fnums:
            self.dataset.extend(load_fire(fnum, split=split))
        print(len(self.dataset))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

trainset = FireDataset(split="Train")
trainset, valset = torch.utils.data.random_split(trainset, [0.9,0.1])
testset = FireDataset(split="Test")
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

In [ ]:
print(trainset)

In [ ]:
import torch
import torch.nn as nn

class FuelEmbeddings(nn.Module):
    def __init__(self, embedding_dim):
        super(FuelEmbeddings, self).__init__()

        unique_values = [0, 1, 2, 3, 4, 7, 13, 31, 101, 425, 635, 650, 665]
        self.unique_values = torch.tensor(unique_values).to(device)  # Unique values in the categorical feature
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(num_embeddings=len(unique_values), embedding_dim=embedding_dim)

    def forward(self, categorical_feature):
        # (B,H,W) -> (B,H,W,U) wher U is unique values count
        mask = categorical_feature.unsqueeze(-1) == self.unique_values
        matching_indices = torch.argmax(mask.int(), dim=-1)

        # Apply embedding and reshape
        # (B,H,W,U) -> (B,H,W,6) -> (B,6,H,W) in default setting
        embedded_fuel = self.embedding(matching_indices)
        embedded_reshaped_fuel = embedded_fuel.permute(0, 3, 1, 2)

        return embedded_reshaped_fuel

class CNN1(nn.Module):
    def __init__(self, embedding_dim=6, num_features=8):
        super(CNN1, self).__init__()

        self.fuelembedding = FuelEmbeddings(embedding_dim)

        # (266, 433)
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=(embedding_dim+num_features-1), out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(inplace=True)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(num_features=8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        categorical_feature = x[:, 0, :, :]  # Extract the categorical feature
        embedded_fuel = self.fuelembedding(categorical_feature)  # Transform the categorical feature

        # Replace the original categorical feature with the embedded feature
        x = torch.cat((embedded_fuel, x[:, 1:, :, :]), dim=1)

        x = self.conv_block1(x)
        x = self.conv_block2(x)
        out = self.sigmoid(x)

        return out


## Define Loss function

<ins>**Create/define/specify your own loss function here!**<ins>

In [ ]:
import torch
import torch.nn as nn

class IoULoss(nn.Module):
    def __init__(self, threshold=0.5):
        super(IoULoss, self).__init__()
        self.threshold = threshold

    def forward(self, outputs, labels):
        # threshold condition is not differentiable so just use softmaxed data
        # Flatten the tensors
        outputs = outputs.view(-1)
        labels = labels.view(-1)

        # Compute the intersection
        intersection = (outputs * labels).sum()

        # Compute the union
        union = outputs.sum() + labels.sum() - intersection
        iou = intersection / (union + 1e-6)  # Add a small epsilon for numerical stability
        loss = 1 - iou
        return loss

#### Metrics for evaluation

In [ ]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

#### Train function

In [ ]:
# Train
def train(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0
    total_steps = 0
    for i, batch in enumerate(dataloader):
        ft = batch['ft'].to(device).float()
        gt = batch['gt'].to(device).float()

        optimizer.zero_grad()
        output = model(ft).squeeze()

        loss = criterion(output, gt)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        total_steps += 1
    return running_loss/total_steps

#### Eval function

In [ ]:
def eval(model, dataloader):
    model.eval()
    acc = []
    iou = []
    f1 = []
    total_steps = 0
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            ft = batch['ft'].to(device)
            gt = torch.flatten(batch['gt'])

            output = torch.flatten(model(ft)).squeeze().cpu()
            output = (output > 0.5)

            acc.append(accuracy_score(gt, output))
            iou.append(jaccard_score(gt, output))
            f1.append(f1_score(gt, output))
            total_steps += 1
    return sum(acc)/total_steps, sum(iou)/total_steps, sum(f1)/total_steps

#### Inference function

Saves the inference results to a submission file!

In [ ]:
def inference(model, dataloader):
    model.eval()
    with torch.no_grad():
        cfire = torch.zeros(target_shape)
        for i, day in enumerate(dataloader):
            ft = day['ft'].to(device)

            # Create the submission file after 10 days
            if i > 9:
                cfire = torch.logical_or(output, cfire) # define the cumulative fire
                ft[0][1] = cfire # set the cumulative fire for the next input
                ft[0][2] = output # set the next step fire for the next input
            else:
                cfire = ft[0][1]

            output = model(ft)
            output = (output > 0.5)

    # Save the cumulative fire
    pred = cfire.cpu().squeeze().numpy()
    save_df = pd.DataFrame(pred)  # convert img data to df
    save_df.to_csv("./output/submission.csv", index_label='row')
    return pred

#### The training/eval/inference loop

<ins>**Define new optimizers here**<ins>

<ins>**Utilize a scheduler here**<ins>

<ins>**Change the learning rate here**<ins>

<ins>**Implement a better early stopping strategy here**<ins>

<ins>**Implement other tricks here (i.e. EMA)**<ins>


In [ ]:
model = CNN1(num_features=8)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = IoULoss()
epochs = 20
best_miou = 0
for e in range(epochs):
    loss = train(model, trainloader, optimizer, criterion)
    aa, miou, mf1 = eval(model,valloader)

    if miou > best_miou:
        best_miou = miou
        cfire = inference(model, testloader)
        e = str(e)+"*"
    print(e, " avg iou loss:{:.3f} avg acc: {:.3f} avg f1: {:.3f} avg iou jaccard score: {:.3f}".format(loss, aa, mf1, miou))



0*  avg iou loss:0.995 avg acc: 0.573 avg f1: 0.018 avg iou: 0.009
1*  avg iou loss:0.993 avg acc: 0.623 avg f1: 0.020 avg iou: 0.010
2*  avg iou loss:0.991 avg acc: 0.903 avg f1: 0.042 avg iou: 0.021
3*  avg iou loss:0.986 avg acc: 0.975 avg f1: 0.123 avg iou: 0.066
4*  avg iou loss:0.956 avg acc: 0.990 avg f1: 0.224 avg iou: 0.126
5*  avg iou loss:0.878 avg acc: 0.994 avg f1: 0.309 avg iou: 0.183
6  avg iou loss:0.843 avg acc: 0.994 avg f1: 0.284 avg iou: 0.167
7*  avg iou loss:0.833 avg acc: 0.994 avg f1: 0.317 avg iou: 0.189
8  avg iou loss:0.832 avg acc: 0.994 avg f1: 0.312 avg iou: 0.187
9*  avg iou loss:0.829 avg acc: 0.994 avg f1: 0.331 avg iou: 0.199
10*  avg iou loss:0.828 avg acc: 0.994 avg f1: 0.342 avg iou: 0.206
11*  avg iou loss:0.825 avg acc: 0.994 avg f1: 0.347 avg iou: 0.210
12  avg iou loss:0.824 avg acc: 0.994 avg f1: 0.333 avg iou: 0.200
13  avg iou loss:0.822 avg acc: 0.994 avg f1: 0.346 avg iou: 0.210
14*  avg iou loss:0.826 avg acc: 0.994 avg f1: 0.361 avg iou: 

## Other Ideas to implement!

<ins>**Ensemble learning - voting**<ins>

<ins>**Implement hot spot data pipeline**<ins>

<ins>**Make better use of temporal data**<ins>

<ins>**Get creative!**<ins>